In [1]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from Total_caller import caller
from flask import Flask,jsonify
app = Flask(__name__)


with open("model.pickle","rb") as f:
    model=pickle.load(f)
    
    
feat_cols=[
        'Temperature',
        'Humidity',
        'Mean Sea Level Pressure  [MSL]',
        'Wind Speed  [10 m above gnd]',
        'High Cloud Cover  [high cld lay]',
        'Wind Direction  [10 m above gnd]',
        'Wind Gust  [sfc]'
]

grid_dev=[(1,3),(0,0),(3,2),(2,1)]
eucledine=lambda a,b,c,d:round(((a-c)**2+(b-d)**2)**0.5,2)

def get_contribution_vec(x,y):
    try:
        device=grid_dev.index((x,y))
        #print('It is a device')
        vec=[0,0,0,0]
        vec[device]=1
    except ValueError:
        #print('Not a device')
        vec=[0,0,0,0]
        for i,(a,b) in enumerate(grid_dev):
            vec[i]=eucledine(a,b,x,y)
    return vec


grid=[]
for i in range(4):
    for j in range(4):
        grid.append(get_contribution_vec(i,j))



def Interpolate(dev,rc):

    #interpolating w.r.to distance vec for each point.

    interpol=[]
    for g in grid:
        x=1/np.array(g).reshape(-1,1)
        x[x == np.inf] = 0

        interpol.append((x*dev).sum(axis=0)/np.sum(x))

    #Interpolated features

    grid_feat=pd.DataFrame(interpol)[feat_cols]

    #RandomForest Predictions

    grid_wise_pred=rc.predict(grid_feat)
    
    #plot the heatmap for whole grid
    """
    fig=plt.figure()
    ax=fig.add_subplot(111)
    sns.heatmap(grid_wise_pred.reshape(4,4),vmin=1,vmax=3,cmap=['green','orange','brown'],annot=True,
                linewidths=1,linecolor='grey',ax=ax)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    #plt.tight_layout()
    ax.set_xticks([])
    ax.set_yticks([])

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=15, axis='both', which='both', length=0)
    cbar.set_ticks([1,2,3])
    cbar.set_ticklabels(['AQI_c1','AQI_c2','AQI_c3'])
    
    #fig.savefig('./logs/Interplolate/day_{}_hour_{}.png'.format(day,hour))
    """
    return grid_wise_pred

#output=Interpolate(df,model)
#print(output)

/home/abhijit/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abhijit/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abhijit/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abhijit/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.23.1. Th

In [2]:
@app.route("/")
def home():
    return "Home"

@app.route('/<date>')
def fun(date='2020-12-19 11'):
    caller(date)
    df=pd.read_csv('Final.csv')[feat_cols] #crower tar output...
    pred=Interpolate(df,model)
    dev_loc=list(range(16))
    dic=dict(zip(dev_loc,pred))
    return jsonify(dic)

In [3]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Dec/2020 13:21:37] "GET / HTTP/1.1" 200 -


Device-4.xls
Device-7.xls
Device-5.xls
Device-6.xls
Device-2.xls
Device-1.xls
Device-3.xls
dataexport_20201224T202439.xlsx
B
C
C
C
C
C
C
C
A
dataexport_20201226T075155.xlsx
dataexport_20201226T075155.xlsx
B
C
Downloading
C
C
C
C
C
C
A
dataexport_20201226T075318.xlsx
Downloading


[2020-12-26 13:24:43,652] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-e5184c66c2b3>", li

finished
finished
Device-6.xls.crdownload
Device-4.xls
Device-7.xls
Device-4.xls.crdownload
Device-5.xls
Device-6.xls
Device-2.xls
Device-2 (1).xls
Device-5 (1).xls
Device-3 (1).xls
Device-1.xls
Device-1.xls.crdownload
Device-3.xls
Device-7 (1).xls
Device-6.xls.crdownload
Device-4.xls
Device-7.xls
Device-4.xls.crdownload
Device-5.xls
Device-6.xls
Device-2.xls
Device-2 (1).xls
Device-5 (1).xls
Device-3 (1).xls
Device-1.xls
Device-1.xls.crdownload
Device-3.xls
Device-7 (1).xls
2020-12-19 09
IOT_data/Device-1.xls
IOT_data/Device-2.xls
IOT_data/Device-3.xls
IOT_data/Device-4.xls


/home/abhijit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in true_divide
127.0.0.1 - - [26/Dec/2020 13:25:06] "GET /2020-12-19%2009 HTTP/1.1" 200 -


  Humidity Temperature           Time
1       49     16.8333  2020-12-19 09
2      NaN         NaN  2020-12-19 09
3  43.9091     17.6494  2020-12-19 09
4       49          11  2020-12-19 09
